# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-12 03:45:39] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=36258, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=652497306, constrained_json_whitespace_pattern

[2025-07-12 03:45:50] Attention backend not set. Use fa3 backend by default.
[2025-07-12 03:45:50] Init torch distributed begin.


[2025-07-12 03:45:51] Init torch distributed ends. mem usage=0.00 GB


[2025-07-12 03:45:52] Load weight begin. avail mem=78.50 GB
[2025-07-12 03:45:52] The weight of LmHead is not packed


[2025-07-12 03:45:52] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-07-12 03:45:57] INFO:     Started server process [580471]
[2025-07-12 03:45:57] INFO:     Waiting for application startup.
[2025-07-12 03:45:57] INFO:     Application startup complete.
[2025-07-12 03:45:57] INFO:     Uvicorn running on http://0.0.0.0:36258 (Press CTRL+C to quit)


[2025-07-12 03:45:58] INFO:     127.0.0.1:58594 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-12 03:45:58] INFO:     127.0.0.1:58606 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-12 03:45:58] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:45:58.318552


[2025-07-12 03:45:59] INFO:     127.0.0.1:58620 - "POST /generate HTTP/1.1" 200 OK
[2025-07-12 03:45:59] The server is fired up and ready to roll!


Server started on http://localhost:36258


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-12 03:46:03] INFO:     127.0.0.1:35082 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-07-12 03:46:03] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:03.112228


[2025-07-12 03:46:03] INFO:     127.0.0.1:35086 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-07-12 03:46:03] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, #token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:03.417761
[2025-07-12 03:46:03] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.84, #queue-req: 0, timestamp: 2025-07-12T03:46:03.485129


[2025-07-12 03:46:03] INFO:     127.0.0.1:35098 - "POST /generate HTTP/1.1" 200 OK


[2025-07-12 03:46:03] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 73, #token: 73, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:03.789944
[2025-07-12 03:46:03] Decode batch. #running-req: 1, #token: 103, token usage: 0.01, cuda graph: False, gen throughput (token/s): 104.63, #queue-req: 0, timestamp: 2025-07-12T03:46:03.867432


[2025-07-12 03:46:04] INFO:     127.0.0.1:35114 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-07-12 03:46:04] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, #token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:04.190509
[2025-07-12 03:46:04] INFO:     127.0.0.1:35122 - "POST /generate HTTP/1.1" 200 OK
[2025-07-12 03:46:04] Prefill batch. #new-seq: 1, #new-token: 2, #cached-token: 31, #token: 31, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:04.216468
[2025-07-12 03:46:04] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 31, #token: 33, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-12T03:46:04.217420
[2025-07-12 03:46:04] INFO:     127.0.0.1:35130 - "POST /generate HTTP/1.1" 200 OK


[2025-07-12 03:46:04] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, #token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:04.254009
[2025-07-12 03:46:04] Decode batch. #running-req: 1, #token: 50, token usage: 0.00, cuda graph: False, gen throughput (token/s): 90.13, #queue-req: 0, timestamp: 2025-07-12T03:46:04.311219


[2025-07-12 03:46:04] Decode batch. #running-req: 1, #token: 90, token usage: 0.00, cuda graph: False, gen throughput (token/s): 110.14, #queue-req: 0, timestamp: 2025-07-12T03:46:04.674401
[2025-07-12 03:46:04] INFO:     127.0.0.1:35134 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-07-12 03:46:04] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, #token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:04.840609
[2025-07-12 03:46:04] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, #token: 49, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-12T03:46:04.841710


[2025-07-12 03:46:05] Decode batch. #running-req: 2, #token: 100, token usage: 0.00, cuda graph: False, gen throughput (token/s): 114.52, #queue-req: 0, timestamp: 2025-07-12T03:46:05.215797


[2025-07-12 03:46:05] Decode batch. #running-req: 2, #token: 180, token usage: 0.01, cuda graph: False, gen throughput (token/s): 209.77, #queue-req: 0, timestamp: 2025-07-12T03:46:05.597163


[2025-07-12 03:46:05] Decode batch. #running-req: 2, #token: 260, token usage: 0.01, cuda graph: False, gen throughput (token/s): 210.90, #queue-req: 0, timestamp: 2025-07-12T03:46:05.976498


[2025-07-12 03:46:06] INFO:     127.0.0.1:35142 - "POST /generate HTTP/1.1" 200 OK
[2025-07-12 03:46:06] Decode batch. #running-req: 1, #token: 192, token usage: 0.01, cuda graph: False, gen throughput (token/s): 173.23, #queue-req: 0, timestamp: 2025-07-12T03:46:06.351732


[2025-07-12 03:46:06] INFO:     127.0.0.1:35156 - "POST /generate HTTP/1.1" 200 OK
[2025-07-12 03:46:06] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 39, #token: 39, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:06.446976


[2025-07-12 03:46:06] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, cuda graph: False, gen throughput (token/s): 65.45, #queue-req: 0, timestamp: 2025-07-12T03:46:06.962843


[2025-07-12 03:46:07] Decode batch. #running-req: 1, #token: 416, token usage: 0.02, cuda graph: False, gen throughput (token/s): 109.87, #queue-req: 0, timestamp: 2025-07-12T03:46:07.326925


[2025-07-12 03:46:07] Decode batch. #running-req: 1, #token: 456, token usage: 0.02, cuda graph: False, gen throughput (token/s): 110.40, #queue-req: 0, timestamp: 2025-07-12T03:46:07.689236


[2025-07-12 03:46:08] Decode batch. #running-req: 1, #token: 496, token usage: 0.02, cuda graph: False, gen throughput (token/s): 110.40, #queue-req: 0, timestamp: 2025-07-12T03:46:08.051561
[2025-07-12 03:46:08] INFO:     127.0.0.1:35166 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-07-12 03:46:08] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, #token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:08.243574


[2025-07-12 03:46:11] INFO:     127.0.0.1:35174 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-07-12 03:46:11] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, #token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:11.811012
[2025-07-12 03:46:11] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, cuda graph: False, gen throughput (token/s): 10.44, #queue-req: 0, timestamp: 2025-07-12T03:46:11.884008


[2025-07-12 03:46:12] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, cuda graph: False, gen throughput (token/s): 103.44, #queue-req: 0, timestamp: 2025-07-12T03:46:12.270690


[2025-07-12 03:46:12] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, cuda graph: False, gen throughput (token/s): 102.18, #queue-req: 0, timestamp: 2025-07-12T03:46:12.662168


[2025-07-12 03:46:13] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, cuda graph: False, gen throughput (token/s): 103.99, #queue-req: 0, timestamp: 2025-07-12T03:46:13.046814
[2025-07-12 03:46:13] INFO:     127.0.0.1:35178 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-07-12 03:46:13] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, #token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:13.064921
[2025-07-12 03:46:13] INFO:     127.0.0.1:35458 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:00,  8.15it/s]

 67%|██████▋   | 2/3 [00:00<00:00, 12.91it/s]

100%|██████████| 3/3 [00:00<00:00, 21.23it/s]

[2025-07-12 03:46:13] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 17, #token: 17, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:13.089312
[2025-07-12 03:46:13] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, #token: 26, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-12T03:46:13.090323
[2025-07-12 03:46:13] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 19, #token: 37, token usage: 0.00, #running-req: 2, #queue-req: 0, timestamp: 2025-07-12T03:46:13.118955
[2025-07-12 03:46:13] INFO:     127.0.0.1:35484 - "POST /generate HTTP/1.1" 200 OK
[2025-07-12 03:46:13] INFO:     127.0.0.1:35498 - "POST /generate HTTP/1.1" 200 OK
[2025-07-12 03:46:13] INFO:     127.0.0.1:35470 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-07-12 03:46:13] INFO:     127.0.0.1:35512 - "POST /generate HTTP/1.1" 200 OK
[2025-07-12 03:46:13] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, #token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:13.238298


The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-07-12 03:46:13] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, #token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:13.328606
[2025-07-12 03:46:13] INFO:     127.0.0.1:35522 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-12 03:46:13] Child process unexpectedly failed with exitcode=9. pid=580759


[2025-07-12 03:46:13] Child process unexpectedly failed with exitcode=9. pid=580625


## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-12 03:46:25] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=33439, nccl_port=None, mem_fraction_static=0.7866, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=576369942, constrained_json_whitespa

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-12 03:46:27] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-12 03:46:28] Inferred chat template from model path: qwen2-vl


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-12 03:46:37] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-12 03:46:37] Attention backend not set. Use flashinfer backend by default.
[2025-07-12 03:46:37] Init torch distributed begin.
[2025-07-12 03:46:37] Init torch distributed ends. mem usage=0.00 GB


...

[2025-07-12 03:46:44] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=128000, available_gpu_mem=60.85 GB


[2025-07-12 03:46:45] INFO:     Started server process [581539]
[2025-07-12 03:46:45] INFO:     Waiting for application startup.
[2025-07-12 03:46:45] INFO:     Application startup complete.
[2025-07-12 03:46:45] INFO:     Uvicorn running on http://0.0.0.0:33439 (Press CTRL+C to quit)
[2025-07-12 03:46:45] INFO:     127.0.0.1:58774 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-12 03:46:46] INFO:     127.0.0.1:58780 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-12 03:46:46] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:46.405092


[2025-07-12 03:46:47] INFO:     127.0.0.1:58782 - "POST /generate HTTP/1.1" 200 OK
[2025-07-12 03:46:47] The server is fired up and ready to roll!


Server started on http://localhost:33439


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-12 03:46:50] INFO:     127.0.0.1:58796 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-07-12 03:46:51] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-12T03:46:51.599861


[2025-07-12 03:46:52] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: False, gen throughput (token/s): 5.32, #queue-req: 0, timestamp: 2025-07-12T03:46:52.438063


[2025-07-12 03:46:53] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: False, gen throughput (token/s): 58.47, #queue-req: 0, timestamp: 2025-07-12T03:46:53.122183


[2025-07-12 03:46:53] INFO:     127.0.0.1:58804 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)

[2025-07-12 03:46:53] Child process unexpectedly failed with exitcode=9. pid=581822
